In [2]:
%pip install pandas sqlalchemy pyodbc

  Using cached pandas-2.3.3-cp314-cp314-win_amd64.whl.metadata (19 kB)
  Using cached sqlalchemy-2.0.45-cp314-cp314-win_amd64.whl.metadata (9.8 kB)
  Using cached pyodbc-5.3.0-cp314-cp314-win_amd64.whl.metadata (2.8 kB)
  Using cached numpy-2.4.1-cp314-cp314-win_amd64.whl.metadata (6.6 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.3-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached greenlet-3.3.0-cp314-cp314-win_amd64.whl.metadata (4.2 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
Using cached pandas-2.3.3-cp314-cp314-win_amd64.whl (11.1 MB)
Using cached sqlalchemy-2.0.45-cp314-cp314-win_amd64.whl (2.1 MB)
Using cached pyodbc-5.3.0-cp314-cp314-win_amd64.whl (72 kB)
Using cached greenlet-3.3.0-cp314-cp314-win_amd64.whl (305 kB)
Using cached numpy-2.4.1-cp314-cp314-win_amd64.whl (12.4 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached typing_extensions-4.15.0-py3-none-any.whl (44 k


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import sqlalchemy as sal
from sqlalchemy import create_engine

In [11]:
from sqlalchemy import create_engine, text

engine = create_engine(
    "mssql+pyodbc://@localhost/stock_analytics"
    "?driver=ODBC+Driver+17+for+SQL+Server"
    "&trusted_connection=yes"
    "&TrustServerCertificate=yes"
)

with engine.connect() as conn:
    result = conn.execute(text("SELECT 1"))
    print(result.fetchone())


(1,)


C:\Users\Admin\AppData\Local\Temp\ipykernel_39492\4185214091.py:10: SAWarning: Unrecognized server version info '17.0.1000.7'.  Some SQL Server features may not function properly.
  with engine.connect() as conn:


In [12]:
csv_table_map = {
    "raw_data/share.csv": "shares",
    "raw_data/share_lookup.csv": "share_lookup",
    "raw_data/share_trade.csv": "share_trade",
}

In [17]:
for csv_file, table_name in csv_table_map.items():
    try:
        print(f"Loading {csv_file} → staging.{table_name}")

        df = pd.read_csv(csv_file)

        # Standardize column names
        df.columns = [c.lower() for c in df.columns]

        # Convert timestamps if present
        if "ts" in df.columns:
            df["ts"] = pd.to_datetime(df["ts"], dayfirst=True, errors="coerce")
        
        if "vol" in df.columns:
            df["vol"] = pd.to_numeric(df["vol"], errors="coerce")

        df.to_sql(
            name=table_name,
            con=engine,
            schema="staging",
            if_exists="append",   # IMPORTANT
            index=False,
            chunksize=200,
            method="multi"
        )

        print(f"✔ Loaded staging.{table_name}\n")

    except Exception as e:
        print(f"❌ Failed loading {csv_file}: {e}\n")


Loading raw_data/share.csv → staging.shares
✔ Loaded staging.shares

Loading raw_data/share_lookup.csv → staging.share_lookup
✔ Loaded staging.share_lookup

Loading raw_data/share_trade.csv → staging.share_trade
✔ Loaded staging.share_trade

